## Contexto do projeto:
### Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

____________

# ***Instalando Biblioteca***

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00


# ***Bibliotecas***

In [ ]:
import pandas as pd
import requests
import json
import openai

# ***Importando e Lendo Dataset***

In [ ]:
df = pd.read_csv('usuario_id.csv')

df

UserID
0         1
1         2
2         3
3         4
4         5
..      ...
245     246
246     247
247     248
248     249
249     250

[250 rows x 1 columns]

In [ ]:
user_ids = df['UserID'].tolist()
print(user_ids)

# ***Extração***

Criando variável para a url

In [ ]:
etl_api_url = 'https://sdw-2023-prd.up.railway.app'

Recuperando user_ids pela API


In [ ]:
def get_user(id):
  response = requests.get(f'{etl_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

In [ ]:
users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

# ***Transformação***

API Key

In [ ]:
openai.api_key = open_api_key

Chamando a API

In [ ]:
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

# ***Load (Carga)***

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")